# Wildfire Proximity Computation Example
This Jupyter notebook contains example code that illustrate how to perform some basic geodetic computations related to the Wildfire course project.  This notebook should provide enough information to complete the Wildfire assignment.

The complete [Wildfire dataset](https://www.sciencebase.gov/catalog/item/61aa537dd34eb622f699df81) can be retrieved from a US government repository. I have noticed that the repository is sometimes "down" and does not respond. It probably makes sense to get the dataset as soon as possible.

This notebook has dependencies on [Pyproj](https://pyproj4.github.io/pyproj/stable/index.html), the [geojson](https://pypi.org/project/geojson/) module and on the wildfire user module. Pyproj and geojson can be installed via pip. The wildfire user module should be downloaded from the course website, unzipped, and moved into the folder pointed to by your PYTHONPATH system variable.

### License
This code example was developed by Dr. David W. McDonald for use in DATA 512, a course in the UW MS Data Science degree program. This code is provided under the [Creative Commons](https://creativecommons.org) [CC-BY license](https://creativecommons.org/licenses/by/4.0/). Revision 1.1 - August 16, 2024

### Preliminaries
First we start with some imports and some constant definitions.

In [1]:
#
#    IMPORTS
# 

#    Import some standard python modules
import os, json, time
#
#    The module pyproj is a standard module that can be installed using pip or your other favorite
#    installation tool. This module provides tools to convert between different geodesic coordinate systems
#    and for calculating distances between points (coordinates) in a specific geodesic system.
#
from pyproj import Transformer, Geod
#
#    The 'wildfire' module is a user module. This module is available from the course website. The module
#    includes one object, a Reader, that can be used to read the GeoJSON files associated with the
#    wildefire dataset. The module also contains a sample datafile that is GeoJSON compliant and that
#    contains a small number of wildfires extracted from the main wildfire dataset.
#    
#
#    There is a GeoJSON reader that you might try if you wanted to read the data. It has its own quirks.
#    There will be an example below that reads the sample file "Wildfire_short_sample.json"
#    

import numpy as np

import pandas as pd

import ast # used to convert a string to a list of lists

## Example 4. Convert points between geodetic coordinate systems

One of the constraints in doing geodetic computations is that most of the time we need to have our points (the coordinates for places) in the same geographic coordinate system. There are tons and tons of coordinate systems. You can find descriptions of many of them at [EPSG.io](https://epsg.io).

Looking at the wildfire header information, you can find this in the output of Example 1 (above), we can see fields named "geometryType" and "spatialReference". This looks like:

        "geometryType": "esriGeometryPolygon",
        "spatialReference": {
            "wkid": 102008,
            "latestWkid": 102008
        },

This indicates that the geometry of our wildfire data are generic polygons and that they are expressed in a coordinate system with the well-known ID (WKID) 102008. This coordinate system is also known as [ESRI:102008](https://epsg.io/102008)

If you look back at Example 2 (above), you might have wondered about the line of code that says:

    geocalc = Geod(ellps='WGS84')         # Use WGS84 ellipsoid representation of the earth

That string, 'WGS84', is a representation of the earth, that also relies on a well known coordinate system that is sometimes called 'decimal degrees' (DD). That decimal degrees system has an official name (or WKID) of [EPSG:4326](https://epsg.io/4326).

For the example below, what we're going to do is take the geometry of a fire feature, extract the largest ring (i.e., the largest boundary of the fire) and convert all of the points in that ring from the ESRI:102008 coordinate system to EPSG:4326 coordinates.

Why? You might ask. Well, it turns out that the easy way to get locations for a city is to make a Google query for the location. That will respond with a decimal degress version of latitued and longitude. So, this allows us to move everything into decimal degress before we perform a distance calcluation.


In [2]:
#
#    Transform feature geometry data
#
#    The function takes one parameter, a list of ESRI:102008 coordinates that will be transformed to EPSG:4326
#    The function returns a list of coordinates in EPSG:4326
def convert_ring_to_epsg4326(ring_data=None):
    converted_ring = list()
    #
    # We use a pyproj transformer that converts from ESRI:102008 to EPSG:4326 to transform the list of coordinates
    to_epsg4326 = Transformer.from_crs("ESRI:102008","EPSG:4326")
    # We'll run through the list transforming each ESRI:102008 x,y coordinate into a decimal degree lat,lon
    for coord in ring_data:
        lat,lon = to_epsg4326.transform(coord[0],coord[1])
        new_coord = lat,lon
        converted_ring.append(new_coord)
    return converted_ring

## Example 5. Compute distance between a place and a wildfire

The basic problem is knowing how far away a fire is from some location (like a city). One issue is that fires are irregularly shaped so the actual answer to that is a bit dependent upon the exact shape and how you want to think about the notion of 'distance'. For example, should we just find the closest point on the perimiter of a fire and call that the distance? Maybe we should find the centroid of the region, identify that as a geolocation (coordinate) and then calculate the distance to that? We can come up with numerous other ways.

The first bit of code finds the point on the perimiter with the shortest distance to the city (place) and returns the distance as well as the lat,lon of the perimeter point.

The second bit of code calculates the average distance of all perimeter points to the city (place) and returns that average as the distance. This is not quite what the centroid would be, but it is probably fairly close.

These are two reasonable ways to think about possible distance to a fire. But both require computing distance to a whole set of points on the perimeter of a fire.


In [3]:
#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns a list containing the shortest distance to the perimeter and the point where that is
#
def shortest_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    closest_point = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        # convert the distance to miles
        distance_in_miles = d[2]*0.00062137
        # if it's closer to the city than the point we have, save it
        if not closest_point:
            closest_point.append(distance_in_miles)
            closest_point.append(point)
        elif closest_point and closest_point[0]>distance_in_miles:
            closest_point = list()
            closest_point.append(distance_in_miles)
            closest_point.append(point)
    return closest_point



#    
#    The function takes two parameters
#        A place - which is coordinate point (list or tuple with two items, (lat,lon) in decimal degrees EPSG:4326
#        Ring_data - a list of decimal degree coordinates for the fire boundary
#
#    The function returns the average miles from boundary to the place
#
def average_distance_from_place_to_fire_perimeter(place=None,ring_data=None):
    # convert the ring data to the right coordinate system
    ring = convert_ring_to_epsg4326(ring_data)    
    # create a epsg4326 compliant object - which is what the WGS84 ellipsoid is
    geodcalc = Geod(ellps='WGS84')
    # create a list to store our results
    distances_in_meters = list()
    # run through each point in the converted ring data
    for point in ring:
        # calculate the distance
        d = geodcalc.inv(place[1],place[0],point[1],point[0])
        distances_in_meters.append(d[2])
    #print("Got the following list:",distances_in_meters)
    # convert meters to miles
    distances_in_miles = [meters*0.00062137 for meters in distances_in_meters]
    # the esri polygon shape (the ring) requires that the first and last coordinates be identical to 'close the region
    # we remove one of them so that we don't bias our average by having two of the same point
    distances_in_miles_no_dup = distances_in_miles[1:]
    # now, average miles
    average = sum(distances_in_miles_no_dup)/len(distances_in_miles_no_dup)
    return average



The function below json_to_dataframe takes in a file path of a JSON file and loads it in as a dataframe. I chose to load in the JSON file because I'm familiar with manipulating dataframes more than I am with JSON files. Additionally, it's easier to periodically save a dataframe and re-load it if I want to pause my work and come back to it.

This function also contains a line which removes an artifact from the column names where they're all preceded by either "geometry" or "attributes"

It also creates a column called "Shape_Area_miles" which converts "Shape_Area" from square meters to square miles. I used this since the "distance_from_chosen_city" is in miles. However, because the smoke estimate uses arbitrary units - this could be omitted.

In [4]:
def json_to_dataframe(filepath):
    """
    Takes in a location of a JSON file containing information about wildfires. Produces a dataframe containing their predicted ORES scores
    Parameters:
        - filepath: File location of JSON with wildfire information.

    Returns a dataframe with wildfire information and adds "Shape_Area_miles" column.
    """
    with open(filepath, "r") as file:
        data = json.load(file)

    # Assuming the wildfires are stored under a "features" key (like in GeoJSON)
    # Adjust based on the actual JSON structure if different
    features = data["features"]

    # Convert to a DataFrame
    # If properties are nested in "properties" key, adjust accordingly
    df = pd.json_normalize(features)

    # Remove "attributes." and "geometry." prefixes from column names
    df.columns = df.columns.str.replace(r"^(attributes\.|geometry\.)", "", regex=True)

    df["Shape_Area_miles"] = df["Shape_Area"] / 2589988
    return df

The function below, distance_from_fire_to_city is used to compute the distance from our chosen city to each fires centroid (middle) in miles. Initially, the dataframe has rings as string, so it first converts the column to a list of lists, then it takes the largest ring (list element 0), and finally computes the distance from the center of the fire to the city coordinates. For the purposes of this project, it uses my city "Sioux Falls", but the city_list parameter is set to take in any city. 

In [5]:
def distance_from_fire_to_city(df, city_list):
    """
    Takes in a dataframe containing information about wildfires. 
    Produces a dataframe with the rings in the correct variable type, 
    isolates the largest ring as a column, 
    then computes the distance from the fire to the city and appends it to a new column
    Parameters:
        - df: Dataframe with wildfire information
        - city_list: A list where the first element is the name of a city, 
          and the second element is a list of the longitude and latitude of that city

    Returns a dataframe with three new columns, rings_procssed, largest_ring, 
    and average_distance_from_[city]
    """
    # Convert each string in the 'rings' column to an actual nested list, handling NaN values
    df['rings_processed'] = df['rings'].apply(
        lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    # Assuming 'rings_2' contains the nested lists
    df['largest_ring'] = df['rings_processed'].apply(
    lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)

    df[f'average_distance_from_{city_list[0]}'] = df['largest_ring'].apply(
    lambda ring_data: average_distance_from_place_to_fire_perimeter(city_list[1], ring_data) 
                      if ring_data is not None else None)
    return df

The function below extract_and_convert_dates is a helper function that is used specifically to parse the Listed_Fire_Dates column and take out important dates and add them to a new column. It takes in the dataframe, the column name (Listed_Fire_Dates), the regex pattern correspond to a specific important date type, and the new column name. An explanation of why this is used is given below.

In [6]:
# Define a function to extract and convert dates from the 'Listed_Fire_Dates' column
def extract_and_convert_dates(df, column_name, regex_pattern, new_column_name):
    """
    Takes in a dataframe containing information about wildfires. 
    A column of strings that contain important dates, and the pattern
    then extracts the date and adds it to a new column

    Parameters:
        - df: Dataframe with wildfire information
        - column_name: The column to extract values from (Listed_Fire_Dates)
        - regex_pattern: A pattern used to identify when a n important date is coming
        - new_column_name: The name of the column where the dates will be extracted to

    Returns a dataframe with a new column containing desired dates
    """
    # Use .loc to avoid SettingWithCopyWarning
    df.loc[:, new_column_name] = df[column_name].str.extract(regex_pattern)[0]
    df.loc[:, new_column_name] = pd.to_datetime(df.loc[:, new_column_name], errors='coerce').dt.date
    return df

The function below wildfire_dates_extraction handles the extraction of important dates from the Listed_Fire_Dates column. Initially, the values in that column are long strings composed of several date entries. This function extracts important dates and adds them to their own columns. The dates are used to find fires that are between 5/1 and 10/31 for the yearly fire season. 

In [7]:
def wildfire_dates_extraction(df):
    """
    Takes in a dataframe containing information about wildfires. 
    Then extracts important dates from Listed_Fire_Dates column 
    and adds them to their own columns

    Parameters:
        - df: Dataframe with wildfire information

    Returns a dataframe with a new columns containing fire start/end dates.
    """
    # Sample regex patterns for date extraction
    discovery_pattern = r"Listed Wildfire Discovery Date\(s\): (\d{4}-\d{2}-\d{2})"
    prescribed_pattern = r"Listed Prescribed Fire Start Date\(s\): (\d{4}-\d{2}-\d{2})"
    end_pattern = r"Listed Wildfire Controlled Date\(s\): (\d{4}-\d{2}-\d{2})"

    # Extract and convert dates
    extract_and_convert_dates(df, "Listed_Fire_Dates", discovery_pattern, "discovery_date")
    extract_and_convert_dates(df, "Listed_Fire_Dates", prescribed_pattern, "prescribed_start")
    extract_and_convert_dates(df, "Listed_Fire_Dates", end_pattern, "end_date")
    return df

The function below, wildfire_filtration, handles fulfilling the restrictions on the fires: they must be within x miles, they must start within the fire season (5/1-10/31), and they must be within the given time frame. This function is separate, because in another instance you may want the full dataset without these limitations. 

I made the decision that if no dates were given, it was assumed the fire happened during fire season. My rationale is that naturally most fires happen during fire season. Obviously this choice affects my smoke estimate and any future results. It reflects how the sparsity of the data can negatively affect the accuracy of the data.

This function also contains an interesting parameter - prescribed_date. There are many controlled fires started to reduce the severity of wildfires. According to the USDA, they last for only a few days. Therefore, if there's a prescribed fire that starts before 4/25, I assume it to be out by the time fire season starts even if there isn't a listed end date. 

Naturally, this choice affects my smoke estimate, but there isn't really a good way of assessing the impact of these controlled burns, especially when the exact duration is often not given.

In [26]:
def wildfire_filtration(df, city_list, start_year, end_year, start_season, end_season, distance_from_city = 650, prescribed_date = "2021-04-25"):
    """
    Takes in a dataframe containing information about wildfires, 
    also takes in the city, a list of restrictions, 
    and filters your dataframe based on these parameters

    Parameters:
        - df: Dataframe with wildfire information
        - city_list: Information about your city
        - start_year: What year to start including fires
        - end_year: The last year to include fires
        - start_season: What month and day fire season starts
        - end_season: What month and day fire season ends
        - distance_from_city: How many miles a fire 
          must be within to include in the filtered dataset
        - prescribed_date:

    Returns a dataframe with a new columns containing fire start/end dates.
    """
    
    df = df[(df["Fire_Year"] >= start_year) & (df["Fire_Year"] <= end_year)].copy()
    df = df[df[f'average_distance_from_{city_list[0]}'] <= distance_from_city].copy()
    # ensures the date columns are datatime
    df["discovery_date"] = pd.to_datetime(df["discovery_date"], errors='coerce')
    df["end_date"] = pd.to_datetime(df["end_date"], errors='coerce')
    df["prescribed_start"] = pd.to_datetime(df["prescribed_start"], errors='coerce')
    # Define the start and end dates for the filter range within each year
    start_month_day = pd.to_datetime(start_season, format="%m-%d").month * 100 + pd.to_datetime(start_season, format="%m-%d").day
    end_month_day = pd.to_datetime(end_season, format="%m-%d").month * 100 + pd.to_datetime(end_season, format="%m-%d").day

    # Extract month and day from "start_date" for comparison, but only where "start_date" is not NA
    month_day = df["discovery_date"].dt.month * 100 + df["discovery_date"].dt.day
    last_month_day = df["end_date"].dt.month * 100 + df["end_date"].dt.day
    cutoff_date = pd.to_datetime(prescribed_date)  # Replace with the appropriate year if necessary
    # Filter to include only dates between 05-01 and 10-31 or where "start_date" is NA
    df = df[(month_day.between(start_month_day, end_month_day)) | (df["discovery_date"].isna())].copy()
    df = df[(last_month_day >= start_month_day) | (df["end_date"].isna())].copy()

    # Filter to keep only rows where "prescribed_start" is on or after April 25, or where "prescribed_start" is NA
    df = df[(df["prescribed_start"] >= cutoff_date) | (df["prescribed_start"].isna())].copy()
    return df

The function below, calc_smoke_impact, creates a new column for my chosen smoke metric. For this project, I chose to do the area of the fire (in miles) / (distance from the city^2). My logic is that the larger the area of the fire, the smoke produced. Conversely, the further the fire is from the city - the more it dissipates. Since the smoke can go in any direction I chose to square this value to indicate that distance was a very impactful metric. 

Ultimately, this is a crude way of estimating smoke. We're often given very little data on the dates of the fire, how different materials burn differently, how winds/weather were around these days, etc. My estimate attempted to use the data we were most confident in.  

In [31]:
def calc_smoke_impact(df, city_list):
    """
    Takes in a dataframe containing information about wildfires, 
    also takes in the city information

    Parameters:
        - df: Dataframe with wildfire information
        - city_list: Information about your city

    Returns a dataframe with a new column called 
    smoke_impact that implements my smoke metric.
    """
    # Ensure avg_dist is not zero to avoid division by zero errors
    df["smoke_impact"] = df["Shape_Area_miles"] / (df[f'average_distance_from_{city_list[0]}'] ** 2)
    return df

The cells below handle loading and modifying the dataframe. I intentionally made them separate and the functions modular so that at any point you could save the dataframe if you wanted to analyze the dataframe at a particular step.

The cell below loads the JSON into a dataframe

In [10]:
wildfire_raw = json_to_dataframe("data/raw/USGS_Wildland_Fire_Combined_Dataset.json")

The cell below creates my Sioux Falls variable with the city name and the coordinates. 

It then computes the distance from each fire to the coordinates provided. 

In [11]:
Sioux_Falls_coordinates = ['Sioux_Falls', [43.54, -96.73]]
wildfire_with_distance = distance_from_fire_to_city(wildfire_raw, Sioux_Falls_coordinates)

The cell below extracts the dates from the Listed_Fire_Dates column

In [12]:
wildfire_distance_dates = wildfire_dates_extraction(wildfire_with_distance)

The cell below takes in a series of restrictions on the fire in order to filter out unneeded fires. 

Interestingly, while we use a 650 mile radius for most of the analysis, one of the plots wants us to look up to 1800 miles. So the data uses that 1800 miles and then in future analysis is dropped down to 650 miles.

In [27]:
wildfire_filtered = wildfire_filtration(wildfire_distance_dates, Sioux_Falls_coordinates, 1961, 2021, "05-01", "10-31", distance_from_city=1800)

C:\Users\PC\AppData\Local\Temp\ipykernel_18948\1558897530.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df = df[(last_month_day >= start_month_day) | (df["end_date"].isna())].copy()


The cell below calculates my chosen smoke metric

In [32]:
wildfire_filtered_smoke = calc_smoke_impact(wildfire_filtered, Sioux_Falls_coordinates)

The cell below saves this data to the desired location with the desired name. Obviously this could be changed to any location a researcher would want or to any name they would want. This cell can also be executed at other steps to get the intermediate data.

In [33]:
wildfire_filtered_smoke.to_csv("data/wildfire_processed.csv")